<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumToyAlgorithm_Deutsch'sAlgorithm_01_ChatGPT_mkbahk_20250331.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-aer==0.15.1
%pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.1 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=afd5dd153c14ae8ee337c927d9370c1325e672ae79463b004df3cbb2ee695be4
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 68.4 MB/s eta 0:00:00
  Cloning https://github.com/qisk

In [2]:
%pip show qiskit
%pip show qiskit-aer

Name: qiskit
Version: 1.2.4
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-textbook
Name: qiskit-aer
Version: 0.15.1
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, psutil, qiskit, scipy
Required-by: 


In [3]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_state_city, plot_bloch_multivector
from qiskit_textbook.tools import array_to_latex
from IPython.display import display, Math
import numpy as np
from qiskit.quantum_info import Statevector

In [4]:
# Deutsch 알고리즘을 구현하는 함수
def deutsch_algorithm(oracle):
    qc = QuantumCircuit(2, 1)  # 2 qubits, 1 classical bit

    # 초기 상태 설정
    qc.x(1)  # 보조 큐비트 q_1을 |1> 상태로 설정
    qc.h([0, 1])  # 두 큐비트에 아다마르 변환 적용

    # 오라클 적용
    qc.append(oracle.to_instruction(), [0, 1])

    # 아다마르 변환 다시 적용
    qc.h(0)

    # 첫 번째 큐비트 측정
    qc.measure(0, 0)

    return qc
###def

In [6]:
# 상수 함수 오라클 (항상 0 반환)
def constant_oracle():
    qc = QuantumCircuit(2, name="Oracle Uf-Constant")
    return qc  # 아무런 연산도 하지 않음 (항상 0 반환)
###def



In [7]:
# 균형 함수 오라클 (NOT 게이트 적용하여 0과 1을 반반으로 만듦)
def balanced_oracle():
    qc = QuantumCircuit(2, name="Oracle Uf-Balanced")
    qc.cx(0, 1)  # 입력 큐비트(q_0)에 따라 보조 큐비트(q_1) 상태 변경
    return qc
###def

In [5]:
# 시뮬레이터 실행 함수
def run_deutsch_algorithm(oracle_function, oracle_name):
    oracle = oracle_function()
    qc = deutsch_algorithm(oracle)

    # 시뮬레이터 실행
    simulator = Aer.get_backend('qasm_simulator')
    compiled_circuit = transpile(qc, simulator)
    #qobj = assemble(compiled_circuit)
    results = simulator.run(compiled_circuit, shots=1024).result()
    counts = results.get_counts()

    # 결과 출력
    print(f"Oracle: {oracle_name}")
    print("Measurement results:", counts)
    print(qc.draw())

    # 결과 히스토그램 출력
    plot_histogram(counts).show()
###def

In [8]:
# 상수 함수 테스트
run_deutsch_algorithm(constant_oracle, "Constant Function")

Oracle: Constant Function
Measurement results: {'0': 1024}
     ┌───┐     ┌─────────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                    ├┤ H ├┤M├
     ├───┤┌───┐│  Oracle Uf-Constant │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                    ├──────╫─
     └───┘└───┘└─────────────────────┘      ║ 
c: 1/═══════════════════════════════════════╩═
                                            0 


In [9]:
# 균형 함수 테스트
run_deutsch_algorithm(balanced_oracle, "Balanced Function")

Oracle: Balanced Function
Measurement results: {'1': 1024}
     ┌───┐     ┌─────────────────────┐┌───┐┌─┐
q_0: ┤ H ├─────┤0                    ├┤ H ├┤M├
     ├───┤┌───┐│  Oracle Uf-Balanced │└───┘└╥┘
q_1: ┤ X ├┤ H ├┤1                    ├──────╫─
     └───┘└───┘└─────────────────────┘      ║ 
c: 1/═══════════════════════════════════════╩═
                                            0 
